In [ ]:
# Проверка гипотез

In [ ]:
%pip install torchvision

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor    

In [ ]:
class SupervisedKernel(nn.Module):
    def __init__(self, kernel: nn.Module = None, supervisor: nn.Module = None):
        super(SupervisedKernel, self).__init__()
        if(kernel is not None):
            self.kernel = kernel
        else:
            self.kernel = nn.Sequential(nn.Linear(8, 8), nn.ReLU())
        self.superviser = supervisor
        if(self.superviser is not None):
            self.superviser.requires_grad_(False)

    def forward(self, x):
        y = self.kernel(x)
        if(self.superviser is not None):
            target = self.superviser(x)
        else:
            target = y
        return y, target
    
    @classmethod
    def is_supervised(self, module: nn.Module):
        return isinstance(module, SupervisedKernel) and (module.superviser is not None)
    
    def update(self, kernel: nn.Module, retrain: bool = True):
        # прежнее ядро становится супервизором
        self.superviser = self.kernel.requires_grad_(False)
        self.kernel = kernel

    def advise(self, x):
        # TODO метод оценки "надзороного ядра"
        pass